<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Forming a multi-stock portfolio

In [ ]:
import pandas_datareader.data as web
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline
p = print

In [ ]:
tickers = ['AAPL','IBM', 'MSFT']
df = web.DataReader(tickers,'yahoo','01/01/2010','10/18/2019')['Adj Close']
df.head()

In [ ]:
df.tail()

In [ ]:
# daily returns series
returns = df.pct_change().dropna()
returns.head()

In [ ]:
returns.tail()

In [ ]:
# Manually selected asset weights
weights = np.asarray([0.5,0.3,0.2])

def portofolio_performance(returns, weights):
    mean, std, cov_matrix = returns.mean(), returns.std(), returns.cov()
    pf_weights = weights
    pf_ret = pf_weights.T.dot(mean) * 252
    pf_vol = np.sqrt(pf_weights.T.dot(cov_matrix).dot(pf_weights)) * np.sqrt(252)
    pf_sharpe = pf_ret / pf_vol
    
    columns = ['Ret','Vol','Sharpe'] + [ticker for ticker in tickers]
    data = [pf_ret,pf_vol,pf_sharpe] + [weight for weight in weights]
    
#     myDict = {k:v for (k,v) in zip(columns, data)} 
    series = pd.Series(data=data,index=columns)
    return series

In [ ]:
pf = portofolio_performance(returns, weights)
pf

We want to investigate the portfolio poerformance using different asset weights. Maually changing each weight would be tedious. Instead, we will randomly select the asset weights of each portfolio, then compare the portfolios.

In [ ]:
np.random.seed(123)
n_portfolio = 5
pf_weights = []
for i in range(n_portfolio):
        weights = np.random.random(len(tickers))
        weights /= sum(weights)
        pf_weights.append(weights)
        
pf_weights

In [ ]:
pf1 = portofolio_performance(returns,pf_weights[0])
pf2 = portofolio_performance(returns,pf_weights[1])
pf3 = portofolio_performance(returns,pf_weights[2])
pf4 = portofolio_performance(returns,pf_weights[3])
pf5 = portofolio_performance(returns,pf_weights[4])

In [ ]:
data = {0:pf[:].values,
        1:pf1[:].values,
        2:pf2[:].values,
        3:pf3[:].values,
        4:pf4[:].values,
        5:pf5[:].values}
columns = pf1.index
df = pd.DataFrame(data, columns).T
df

In [ ]:
# The highest Sharpe Ratio portfolio
max_sharpe_pf = df[df['Sharpe'] == df['Sharpe'].max()]
max_sharpe_pf

In [ ]:
# The highest returns portfolio
max_ret_pf = df[df['Ret'] == df['Ret'].max()]
max_ret_pf

In [ ]:
# The lowest volatility portfolio
min_vol_pf = df[df['Vol'] == df['Vol'].min()]
min_vol_pf

In [ ]:
tickers = ['AAPL','IBM', 'MSFT', 'WMT']
df = web.DataReader(tickers,'yahoo','01/01/2010','10/18/2019')['Adj Close']
df.head()

In [ ]:
df.tail()

In [ ]:
# daily returns series
returns = df.pct_change().dropna()
returns.head()

In [ ]:
returns.tail()

In [ ]:
nDays = 1
mean = returns.mean() * nDays
stdev = returns.std() * np.sqrt(nDays)
for ticker, ret, vol in zip(tickers,mean,stdev):
    p(f'{ticker} stock {nDays}-day performance:')
    p(f'The mean daily return is {round(ret,6)}.')
    p(f'The standatd deviation of daily returns (volatitliy) is {round(vol,4)*100}%.\n')

In [ ]:
nDays = 10
mean_10d = returns.mean() * nDays
stdev_10d = returns.std() * np.sqrt(nDays)
for ticker, ret, vol in zip(tickers,mean_10d,stdev_10d):
    p(f'{ticker} stock {nDays}-day performance:')
    p(f'The mean daily return is {round(ret,6)}.')
    p(f'The standatd deviation of daily returns (volatitliy) is {round(vol,4)*100}%.\n')
# for ticker, ret, vol in zip(tickers,mean,stdev):
#     p(f'The {nDays} mean return of {ticker} is {round(ret*nDays,5)*100}%, and {nDays} volatility is {round(vol*np.sqrt(nDays),3)*100}% \n')

In [ ]:
for ticker, var in zip(tickers, returns.var()):
    p(f'The variance of daily returns for {ticker} is {var}.')

In [ ]:
# the covariance of a variable with itself is equal to the variance of that variable
for ticker, cov in zip(tickers,np.cov(returns[tickers])):
    p(f'The covariance of daily returns for {ticker} is {np.cov(returns[ticker])}.')

In [ ]:
cov_matrix = returns.cov()
cov_matrix

In [ ]:
corr_matrix = returns.corr()
corr_matrix

In [ ]:
# portfolio weights
weights = np.asarray([0.5, 0.3, 0.1, 0.1])

def portofolio_performance(returns, weights):
    mean, std, cov_matrix = returns.mean(), returns.std(), returns.cov()
    pf_weights = weights
    pf_ret = pf_weights.T.dot(mean) * 252
    pf_vol = np.sqrt(pf_weights.T.dot(cov_matrix).dot(pf_weights)) * np.sqrt(252)
    rf = 0.0003 # risk-free rate of return
    pf_sharpe = (pf_ret - rf) / pf_vol
    
    columns = ['Ret','Vol','Sharpe'] + [ticker for ticker in tickers]
    data = [pf_ret,pf_vol,pf_sharpe] + [weight for weight in weights]
    series = pd.Series(data=data,index=columns)

    return series

In [ ]:
pf_perf = portofolio_performance(returns,weights)
pf_perf

We created a portfolio with manually selected asset weights. Next, we will create five new portfolios. Each new portfolio will be constructed using randomly selected asset weights. Then, we will compare the all of the portfolios.

In [ ]:
np.random.seed(54321)
n_portfolio = 8
pf_weights = []
for i in range(n_portfolio):
        weights = np.random.random(len(tickers))
        weights /= sum(weights)
        pf_weights.append(weights)
        
pf_weights

In [ ]:
pf1 = portofolio_performance(returns,pf_weights[0])
pf2 = portofolio_performance(returns,pf_weights[1])
pf3 = portofolio_performance(returns,pf_weights[2])
pf4 = portofolio_performance(returns,pf_weights[3])
pf5 = portofolio_performance(returns,pf_weights[4])
pf6 = portofolio_performance(returns,pf_weights[5])
pf7 = portofolio_performance(returns,pf_weights[6])
pf8 = portofolio_performance(returns,pf_weights[7])

In [ ]:
pf1

In [ ]:
data = {0:pf_perf[:].values,
        1:pf1[:].values,
        2:pf2[:].values,
        3:pf3[:].values,
        4:pf4[:].values,
        5:pf5[:].values,
        6:pf6[:].values,
        7:pf7[:].values,
        8:pf8[:].values}
columns = pf1.index
df = pd.DataFrame(data, columns).T
df

In [ ]:
# The highest Sharpe Ratio portfolio
max_sharpe_pf = df[df['Sharpe'] == df['Sharpe'].max()]
max_sharpe_pf

In [ ]:
# The highest returns portfolio
max_ret_pf = df[df['Ret'] == df['Ret'].max()]
max_ret_pf

In [ ]:
# The lowest volatility portfolio
min_vol_pf = df[df['Vol'] == df['Vol'].min()]
min_vol_pf

We have created and investigated many portfolios. Each portfolio is made of n-assets stocks, each of different weights. One portfolio was created by manually selecting the weight of each asset. A few portfolios were created by randomly selecting the weights of each asset. Of the portfolios, we found the highest Sharpe Ratio, highest returns, and lowest volatility portfolios. We will create many more portfolios of randomly generated asset weights, and find the best portfolios.